In [1]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.


In [2]:
from http.server import SimpleHTTPRequestHandler, HTTPServer
from urllib import request, error
import pandas as pd
from pycaret.clustering import *
import urllib.parse
import sys

badwords = ['sleep', 'drop', 'uid', 'select', 'waitfor', 'delay', 'system', 'union', 'order by', 'group by']
def ExtractFeatures(path):

    path = urllib.parse.unquote(path)

    badwords_count = 0

    single_q = path.count("'")

    double_q = path.count('"')

    dashes = path.count("--")

    braces = path.count("(")

    spaces = path.count(" ")

    for word in badwords:
        badwords_count += path.count(word)

    lst = [single_q, double_q, dashes, braces, spaces, badwords_count]

    print(lst)

    return pd.DataFrame([lst], columns=['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'badwords'])


In [3]:
http=pd.read_csv(r'C:\Users\Student\Desktop\asdasdas\good&bad.csv')

In [4]:
clu1=setup(data=http,normalize=True,numeric_features=['single_q'	,'double_q'	,'dashes'	,'braces',	'spaces'	,'badwords'], ignore_features=['method','path','body', 'class'])

,Description,Value
0,Session id,6800
1,Original data shape,"(2853, 10)"
2,Transformed data shape,"(2853, 6)"
3,Ignore features,4
4,Numeric features,6
5,Rows with missing values,90.7%
6,Preprocess,True
7,Imputation type,simple
8,Numeric imputation,mean
9,Categorical imputation,mode


In [5]:
kmeans=create_model('kmeans', num_clusters=2)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.9283,780.0446,1.0563,0,0,0


In [ ]:
class SimpleHTTPProxy(SimpleHTTPRequestHandler):
    proxy_routes = {}

    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        # print (self.proxy_routes)
        # print (self.path)
        parts = self.path.split('/')
        print (parts)
        live_data = ExtractFeatures(parts[3])
        result = predict_model(kmeans, data = live_data)
        print(result['Cluster'][0])
        if result['Cluster'][0] == "Cluster 1":
            print('Intrusion Detected')
        # print (parts, len(parts), parts[1])
        if len(parts) > 2:
            # url = self.proxy_routes[parts[2]] + '/' + '/'.join(parts[2:])
            # print(url)
            self.proxy_request('http://' + parts[2] + '/')
        else:
            super().do_GET()

    def proxy_request(self, url):
        # print('proxy req')
        # print(url)

        try:
            response = request.urlopen(url)
        except error.HTTPError as e:
            print('err')
            self.send_response_only(e.code)
            self.end_headers()
            return
        self.send_response_only(response.status)
        for name, value in response.headers.items():
            self.send_header(name, value)
        self.end_headers()
        self.copyfile(response, self.wfile)

SimpleHTTPProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})
with HTTPServer(('127.0.0.1', 8080), SimpleHTTPProxy) as httpd:
    host, port = httpd.socket.getsockname()
    print(f"Listening on http://{host}:{port}")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nKeyboard interrupt received, exiting.")
        sys.exit()



Listening on http://127.0.0.1:8080
['http:', '', 'demo.testfire.net', 'search.jsp?query=+UNION+SELECT+sum%28columnname+%29+from+tablename+--+']
[0, 0, 1, 1, 0, 0]
Cluster 1
Intrusion Detected


127.0.0.1 - - [09/Dec/2023 08:21:50] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:21:50] "CONNECT telemetry.invicti.com:443 HTTP/1.0" 501 -


['http:', '', 'demo.testfire.net', 'search.jsp?query=+UNION+SELECT++tablename+--+']
[0, 0, 1, 0, 0, 0]
Cluster 1
Intrusion Detected


127.0.0.1 - - [09/Dec/2023 08:22:51] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:22:51] "CONNECT telemetry.invicti.com:443 HTTP/1.0" 501 -
127.0.0.1 - - [09/Dec/2023 08:23:10] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:23:10] "CONNECT contile.services.mozilla.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [09/Dec/2023 08:23:10] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:23:10] "CONNECT spocs.getpocket.com:443 HTTP/1.1" 501 -
127.0.0.1 - - [09/Dec/2023 08:23:51] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:23:51] "CONNECT telemetry.invicti.com:443 HTTP/1.0" 501 -
127.0.0.1 - - [09/Dec/2023 08:24:52] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:24:52] "CONNECT telemetry.invicti.com:443 HTTP/1.0" 501 -
127.0.0.1 - - [09/Dec/2023 08:25:52] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [09/Dec/2023 08:25:52] "C